# Brain Tumor Segmentation

This model will segment the tumors from MRI Images and output the location of the brain tumor

In [1]:
!pip install mlflow-skinny

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.2/779.2 kB 43.8 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader, random_split, Dataset
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os
import mlflow

In [3]:
# Data Augmentation
image_transform = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])
])

mask_transform = transforms.Compose([
    transforms.Resize((256,256), interpolation=Image.NEAREST),
    transforms.ToTensor()
])

In [20]:
class SegmentationDataset(Dataset):
    def __init__(self, image_dir, mask_dir, image_transform=None, mask_transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.image_transform = image_transform
        self.mask_transform = mask_transform
        self.images = sorted(os.listdir(image_dir))


    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.images[idx])
        mask_path = os.path.join(self.mask_dir, self.images[idx])

        image = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")

        if self.image_transform:
            image = self.image_transform(image)

        if self.mask_transform:
            mask = self.mask_transform(mask)

        return image, mask

In [21]:
dataset = SegmentationDataset(
    image_dir = "/kaggle/input/brain-tumor-segmentation/images",
    mask_dir = "/kaggle/input/brain-tumor-segmentation/masks",
    image_transform = image_transform,
    mask_transform = mask_transform
)

In [22]:
dataset_size = len(dataset)

train_size = int(0.70 * dataset_size)
val_size = int(0.15 * dataset_size)
test_size = dataset_size - (train_size + val_size)

train_dataset, val_dataset, test_dataset = random_split(
    dataset,
    [train_size, val_size, test_size]
)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [23]:
#Building block of the Double convolution of U net

class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, padding=1),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv(x)

In [33]:
## Unet

class UNet(nn.Module):
    def __init__(self):
        super().__init__()
    
        self.down1 = DoubleConv(3,64)
        self.pool1 = nn.MaxPool2d(2)
    
        self.down2 = DoubleConv(64, 128)
        self.pool2 = nn.MaxPool2d(2)
    
        self.down3 = DoubleConv(128, 256)
        self.pool3 = nn.MaxPool2d(2)
    
        self.down4 = DoubleConv(256, 512)
        self.pool4 = nn.MaxPool2d(2)
    
        self.bottleneck = DoubleConv(512, 1024)
    
        self.up4 = nn.ConvTranspose2d(1024, 512, 2, stride=2)
        self.conv4 = DoubleConv(1024, 512)
    
        self.up3 = nn.ConvTranspose2d(512, 256, 2, stride=2)
        self.conv3 = DoubleConv(512, 256)
    
        self.up2 = nn.ConvTranspose2d(256,128, 2, stride=2)
        self.conv2 = DoubleConv(256, 128)
    
        self.up1 = nn.ConvTranspose2d(128, 64, 2, stride=2)
        self.conv1 = DoubleConv(128, 64)
    
        self.final = nn.Conv2d(64, 1, kernel_size=1)

    def forward(self,x):
        c1 = self.down1(x)
        p1 = self.pool1(c1)

        c2 = self.down2(p1)
        p2 = self.pool2(c2)

        c3 = self.down3(p2)
        p3 = self.pool3(c3)

        c4 = self.down4(p3)
        p4 = self.pool4(c4)

        bn = self.bottleneck(p4)

        u4 = self.up4(bn)
        u4 = torch.cat([u4, c4], dim=1)
        u4 = self.conv4(u4)

        u3 = self.up3(u4)
        u3 = torch.cat([u3, c3], dim=1)
        u3 = self.conv3(u3)

        u2 = self.up2(u3)
        u2 = torch.cat([u2, c2], dim=1)
        u2 = self.conv2(u2)

        u1 = self.up1(u2)
        u1 = torch.cat([u1, c1], dim=1)
        u1 = self.conv1(u1)

        return torch.sigmoid(self.final(u1))

In [37]:
## Training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

mlflow.set_tracking_uri("file:///kaggle/working/mlruns")
mlflow.set_experiment("tumor segmentation with UNet")

params = {
    "epochs": 50,
    "learning_rate":1e-4,
    "batch_size":8
}

In [38]:
## Accuracy models

def dice_score(preds, targets, smooth=1e-6):
    preds = (preds > 0.5).float()
    intersection = (preds * targets).sum()
    return (2. * intersection + smooth) / (preds.sum() + targets.sum() + smooth)

def pixel_accuracy(preds, targets):
    preds = (preds > 0.5).float()
    correct = (preds == targets).sum()
    total = targets.numel()
    return correct / total

def iou_score(preds, targets, smooth=1e-6):
    preds = (preds > 0.5).float()
    intersection = (preds * targets).sum()
    union = preds.sum() + targets.sum() - intersection
    return (intersection + smooth) / (union + smooth)

In [ ]:
## Training loop function
# def train_model(train_loader, val_loader, criterion, optimizer):

with mlflow.start_run():

    mlflow.log_params(params)
    
    model = UNet().to(device)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=params["learning_rate"])
    
    train_dice, val_dice, train_loss, val_loss = [],[],[],[]
    best_val_dice = 0.0
    
    for epoch in range(params["epochs"]):
        model.train()
        running_loss = 0.0
        running_dice = 0.0
        num_batches = 0
    
        for imgs, masks in train_loader:
            imgs, masks = imgs.to(device), masks.to(device)
    
            optimizer.zero_grad()
            outputs = model(imgs)
    
            #Back propergation
            loss = criterion(outputs, masks)
            loss.backward()
            optimizer.step()
    
            running_loss += loss.item()
            running_dice += dice_score(outputs, masks).item()
            num_batches += 1
    
        epoch_train_loss = running_loss/num_batches
        epoch_train_dice = running_dice/num_batches
    
        train_loss.append(epoch_train_loss)
        train_dice.append(epoch_train_dice)
    
        model.eval()
        running_loss = 0.0
        running_dice = 0.0
        num_batches = 0
    
        with torch.no_grad():
            for imgs, masks in val_loader:
                imgs, masks = imgs.to(device), masks.to(device)
                outputs = model(imgs)
    
                loss = criterion(outputs, masks)
                running_loss += loss.item()
    
                running_dice += dice_score(outputs, masks).item()
                num_batches += 1
    
        epoch_val_loss = running_loss/num_batches
        epoch_val_dice = running_dice/num_batches
    
        val_loss.append(epoch_val_loss)
        val_dice.append(epoch_val_dice)
    
        ## saving the best model
        if epoch_val_dice > best_val_dice:
            best_val_dice = epoch_val_dice
            torch.save(model.state_dict(),"best_model.pth")
    
        print(f"Epoch [{epoch+1}/{params["epochs"]}], Tr dice: {100*epoch_train_dice:.2f}%, Val dice: {100*epoch_val_dice:.2f}%, Best Val Acc: {100*best_val_dice:.2f}%")
        mlflow.log_metrics({
            "training_loss":epoch_train_loss,
            "training_dice":epoch_train_dice,
            "validation_loss":epoch_val_loss,
            "validation_dice":epoch_val_dice
        }, step=epoch)
    
    print("Training Complete")
    mlflow.pytorch.log_model(model, artifact_path="model")

Epoch [1/50], Tr dice: 0.17%, Val dice: 0.00%, Best Val Acc: 0.00%
Epoch [2/50], Tr dice: 0.13%, Val dice: 0.01%, Best Val Acc: 0.01%
Epoch [3/50], Tr dice: 11.34%, Val dice: 17.46%, Best Val Acc: 17.46%
Epoch [4/50], Tr dice: 28.20%, Val dice: 31.62%, Best Val Acc: 31.62%
Epoch [5/50], Tr dice: 38.70%, Val dice: 49.94%, Best Val Acc: 49.94%
Epoch [6/50], Tr dice: 46.95%, Val dice: 54.70%, Best Val Acc: 54.70%
Epoch [7/50], Tr dice: 52.22%, Val dice: 60.11%, Best Val Acc: 60.11%
Epoch [8/50], Tr dice: 57.61%, Val dice: 59.53%, Best Val Acc: 60.11%
Epoch [9/50], Tr dice: 60.00%, Val dice: 65.19%, Best Val Acc: 65.19%
Epoch [10/50], Tr dice: 65.82%, Val dice: 59.56%, Best Val Acc: 65.19%
Epoch [11/50], Tr dice: 68.64%, Val dice: 65.36%, Best Val Acc: 65.36%
Epoch [12/50], Tr dice: 71.80%, Val dice: 70.82%, Best Val Acc: 70.82%
Epoch [13/50], Tr dice: 76.17%, Val dice: 67.87%, Best Val Acc: 70.82%
Epoch [14/50], Tr dice: 78.36%, Val dice: 74.06%, Best Val Acc: 74.06%
Epoch [15/50], Tr dic